# 002 Create the Stock Prices Database

This notebook creates and populates the `assets.db` SQLite database with stock tickers and historical OHLC prices from Alpaca (~175 stocks ~NYSE/NASDAQ/AMEX ~`$0`). It uses utility functions from `src/etl/` (~`populate_tickers.py`, `populate_prices.py`, and `utils.py` ~`$0`) to:
- Connect to Alpaca using credentials from `credentials/`.
- Populate `asset_metadata` with active, tradable tickers.
- Populate `asset_prices` with historical data (~2002 to yesterday ~`$0`) and the latest bars.

Progress bars (~via `tqdm` ~`$0`) track each step, and interim print statements are minimized (~focus on timing ~`$0`).

In [2]:
# Import necessary libraries
import sys
import time
from tqdm.notebook import tqdm

# Add project root to sys.path
sys.path.append(r"d:\dev\stat_656_autotrader")

# Import credentials and ETL functions
from credentials import ALPACA_API_KEY, ALPACA_SECRET_KEY, ALPAKA_ENDPOINT_URL
from src.etl.populate_tickers import ensure_database_exists, populate_tickers
# from src.etl import (
#     ensure_database_exists, populate_tickers, ensure_prices_table_exists,
#     get_tickers_from_db, populate_historical_prices, update_latest_prices,
#     connect_to_alpaca
# )

ImportError: cannot import name 'ensure_prices_table_exists' from 'src.etl.populate_prices' (d:\dev\stat_656_autotrader\src\etl\populate_prices.py)

In [ ]:
# Connect to Alpaca
alpaca = connect_to_alpaca(ALPACA_API_KEY, ALPACA_SECRET_KEY, ALPAKA_ENDPOINT_URL)
if not alpaca:
    raise Exception("Failed to connect to Alpaca. Check credentials and network.")

In [ ]:
# Step 1: Ensure database exists and populate tickers
start_time = time.time()
with tqdm(total=2, desc="Database Setup", unit="step") as pbar:
    ensure_database_exists()
    pbar.update(1)
    populate_tickers(alpaca)
    pbar.update(1)
ticker_time = time.time() - start_time

In [ ]:
# Step 2: Populate prices (historical and latest)
start_time = time.time()
with tqdm(total=3, desc="Price Population", unit="step") as pbar:
    ensure_prices_table_exists()
    pbar.update(1)
    tickers_dict = get_tickers_from_db()
    if not tickers_dict:
        raise Exception("No tickers found in asset_metadata.")
    populate_historical_prices(alpaca, tickers_dict)
    pbar.update(1)
    update_latest_prices(alpaca, tickers_dict)
    pbar.update(1)
price_time = time.time() - start_time

In [ ]:
# Display total time taken
total_time = ticker_time + price_time
print(f"Database setup completed in {total_time:.2f} seconds.")
print(f"- Tickers: {ticker_time:.2f} seconds")
print(f"- Prices: {price_time:.2f} seconds")